# Эксперимент с обучением FCRN на малом датасете, random crop

### Датасет

Датасет состоит из 56500 изображений (18800 картинок из исходной коллекции, с каждой из них взято по три случайных кропа). Аугментация не проводилась.

RGB изображения процессились с помощью `keras.applications.resnet50.preprocess_input`. После данного препроцессинга, значения пикселей изображения лежат в пределах примерно от -123 до 150. Глубины подавались в модель в исходном виде в метрах.

### Модель

Архитектура модели взята из статьи: https://arxiv.org/pdf/1606.00373.pdf. Основу модели составляет сверточный энкодинг из ResNet50, предобученной на ImageNet. После этого добавляются 5 Up-projection блоков, описанных в статье. Перед каждым блоком добавляется слой BatchNormalization, после каждого блока - дропаут с коэффициентом 0.5.

Сначала обучаются только разверточные слои (одну эпоху), затем три эпохи - разверточные и последние 3 Resnet-блока, затем размораживаются еще 3 Resnet-блока и проводится одна эпоха обучения, а потом размораживаются все блоки и проводится финальная эпоха.

## Создание модели

In [1]:
import tensorflow as tf
import keras
import numpy as np

Using TensorFlow backend.


In [2]:
assert tf.test.is_gpu_available()

In [3]:
from keras.applications.resnet50 import ResNet50, preprocess_input

In [4]:
model = ResNet50(input_shape=(224, 224, 3), weights='imagenet', include_top=False)

/home/kmouraviev/anaconda/envs/tensorflow/lib/python3.6/site-packages/keras_applications/resnet50.py:263: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [5]:
for layer in model.layers:
    layer.trainable = False

In [6]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [7]:
model.compile(loss='mean_squared_error', optimizer='adam')

In [8]:
resnet_output = model.output

In [9]:
from keras.layers import *

In [10]:
def apply_upprojection(nn):
    n_filters = int(nn.shape[-1])
    batchnorm = BatchNormalization()(nn)
    upsampling = UpSampling2D()(batchnorm)
    conv = Conv2D(n_filters // 2, kernel_size=5, padding='same')(upsampling)
    relu = Activation('relu')(conv)
    conv_3 = Conv2D(n_filters // 2, kernel_size=3, padding='same')(relu)
    conv_proj = Conv2D(n_filters // 2, kernel_size=5, padding='same')(upsampling)
    add = Add()([conv_3, conv_proj])
    relu_2 = Activation('relu')(add)
    dropout = Dropout(0.5)(relu_2)
    return dropout

In [11]:
from keras.layers import *
nn = Conv2D(1024, kernel_size=1, padding='same')(resnet_output)
for k in range(5):
    nn = apply_upprojection(nn)
depth_output = Conv2D(1, kernel_size=3, padding='same')(nn)
depth_output = Reshape((224, 224))(depth_output)

In [12]:
from keras.models import Model
fcrn_model = Model(inputs=model.input, outputs=depth_output)

In [13]:
fcrn_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

##### Загрузка датасета

In [14]:
import h5py

In [15]:
!echo $HDF5_USE_FILE_LOCKING

FALSE


In [16]:
data_file = h5py.File('NYU_dataset_hdf5/data.hdf5', 'r')
rgbs = data_file['data']
depths = data_file['label']

In [17]:
depths = np.array(depths)
depths = depths / depths.max() * 10

In [18]:
rgbs.shape, depths.shape

((56541, 224, 224, 3), (56541, 224, 224))

In [19]:
#rgbs = [np.transpose(image, [1, 2, 0]) for image in rgbs]
rgbs = np.array(rgbs)
rgbs = 2 * rgbs - 1

In [20]:
rgbs = preprocess_input(rgbs)

In [21]:
print(rgbs.min(), rgbs.max(), depths.min(), depths.max())

-122.68 151.061 0.0 10.0


In [22]:
from sklearn.model_selection import train_test_split
rgbs_train, rgbs_val, depths_train, depths_val = train_test_split(rgbs, depths, test_size=0.2)

In [23]:
print(rgbs_train.shape, depths_train.shape, rgbs_val.shape, depths_val.shape)

(45232, 224, 224, 3) (45232, 224, 224) (11309, 224, 224, 3) (11309, 224, 224)


## Обучение

### Сначала только разверточные слои

In [24]:
from keras.optimizers import Adam

In [25]:
fcrn_model.compile(optimizer=Adam(lr=1e-4, decay=1e-4), loss='mean_squared_error', metrics=['mean_squared_error'])

In [26]:
fcrn_model.fit(rgbs_train, depths_train, batch_size=64, epochs=1, validation_data=[rgbs_val, depths_val])

Train on 45232 samples, validate on 11309 samples
Epoch 1/1
45232/45232 [==============================] - 813s 18ms/step - loss: 2.1575 - mean_squared_error: 2.1575 - val_loss: 1.4846 - val_mean_squared_error: 1.4846


In [27]:
fcrn_model.save('model_trained_fcn.hdf5')

### Разморозим 3 resnet-блока

In [ ]:
#fcrn_model = load_model('model_trained_fcn.hdf5')

In [36]:
after_add12 = False
for layer in fcrn_model.layers:
    layer.trainable = after_add12
    if layer.name == 'add_12':
        after_add12 = True
        print(layer)

In [38]:
fcrn_model.compile(optimizer=Adam(lr=1e-5, decay=1e-3), loss='mean_squared_error', metrics=['mean_squared_error'])

In [39]:
fcrn_model.fit(rgbs_train, depths_train, validation_data=[rgbs_val, depths_val], batch_size=32, epochs=3)

Train on 45232 samples, validate on 11309 samples
Epoch 1/5
45232/45232 [==============================] - 748s 17ms/step - loss: 1.1785 - mean_squared_error: 1.1785 - val_loss: 1.1651 - val_mean_squared_error: 1.1651
Epoch 2/5
45232/45232 [==============================] - 738s 16ms/step - loss: 1.0278 - mean_squared_error: 1.0278 - val_loss: 1.1872 - val_mean_squared_error: 1.1872
Epoch 3/5
45232/45232 [==============================] - 738s 16ms/step - loss: 0.9599 - mean_squared_error: 0.9599 - val_loss: 1.1853 - val_mean_squared_error: 1.1853
Epoch 4/5
11008/45232 [======>.......................] - ETA: 8:33 - loss: 0.9154 - mean_squared_error: 0.9154

KeyboardInterrupt: 

In [40]:
fcrn_model.save('model_trained_since_add12.hdf5')

### Разморозим еще 3 resnet-блока

In [41]:
after_add9 = False
for layer in model.layers:
    layer.trainable = after_add9
    if layer.name == 'add_9':
        after_add9 = True
        print(layer)

In [43]:
fcrn_model.fit(rgbs_train, depths_train, validation_data=[rgbs_val, depths_val], batch_size=32, epochs=2)

Train on 45232 samples, validate on 11309 samples
Epoch 1/5
45232/45232 [==============================] - 729s 16ms/step - loss: 0.9093 - mean_squared_error: 0.9093 - val_loss: 1.1891 - val_mean_squared_error: 1.1891
Epoch 2/5
 3904/45232 [=>............................] - ETA: 10:05 - loss: 0.9088 - mean_squared_error: 0.9088

KeyboardInterrupt: 

In [44]:
fcrn_model.save('model_train_since_add9.hdf5')

### Ну и разморозим всё

In [45]:
for layer in model.layers:
    layer.trainable = True

In [46]:
fcrn_model.fit(rgbs_train, depths_train, validation_data=[rgbs_val, depths_val], batch_size=32, epochs=1)

Train on 45232 samples, validate on 11309 samples
Epoch 1/1
45232/45232 [==============================] - 718s 16ms/step - loss: 0.8810 - mean_squared_error: 0.8810 - val_loss: 1.1820 - val_mean_squared_error: 1.1820


In [47]:
fcrn_model.save('model_train_all_layers.hdf5')